# SO, SO2, PPD: gerenciamento de *threads* com *pthreads*

Hélio - DC/UFSCar - 2023



# pthreads

A biblioteca padrão para o gerenciamento de *threads* em sistemas compatíveis com POSIX, como Linux e BSDs, por exemplo, é [pthreads](https://pubs.opengroup.org/onlinepubs/007908775/xsh/pthread.h.html).


## Criação de novas threads

A criação de *threads* é feita com a chamada [pthread_create](https://man7.org/linux/man-pages/man3/pthread_create.3.html)():
```
pthread_create(pthread_t *restrict thread, const pthread_attr_t *restrict attr,
               void *(*start_routine)(void *), void *restrict arg);
```

Alguns aspectos a observar nesta chamada:

* o código de uma *thread* é definido por uma **função** dentro do código do processo.
* A função da *thread* tem um único parâmetro, que é um ponteiro para *void*
* O valor de retorno da *thread* é também um ponteiro para *void*
* ao concluir a execução do código da função especificada, a tarefa criada (thread) termina, sem retornar ao ponto do código que chamou pthread_create()

Embora ter os parâmetros e o valor de retorno definidos desta forma possa parecer algo restritivo, esse modelo oferece grande flexibilidade, já que, em C, pode-se converter esses parâmetros para ponteiros para qualquer tipo de variável, inclusive para estruturas.

O **identificador da nova *thread*** criada é salvo no endereço de memória passado no primeiro parâmetro da chamada.

A criação de *threads* pode ocorrer em qualquer ponto em um programa. Nos casos mais simples, um programa que manipula *threads* vai fazer chamadas à função *pthread_create* para criá-las, a partir da função *main*.

Ah, vale observar o valor de retorno da chamada de criação: **0 = sucesso**, outro valor = código do erro ocorrido.


## Identificador de uma *thread*

Assim como o valor **pid** identifica unicamente um processo para o SO, cada *thread* tem também um identificador único, chamado ***thread ID***. Como mencionado, esse valor é retornado como um dos parâmetros da chamada *pthread_create*. Além disso, uma *thread* pode obter o seu idenficador usando a chamada ***pthread_self***(), que é equivalente à chamada ***getpid***() para processos.

Observe que todas as *threads* de um processo compartilham o mesmo identificador de processo. Assim, a execução da chamada *getpid*() por qualquer *thread* de um processo deve retornar o mesmo valor.

## Encerramento da execução de *threads*

Depois de criar novas *threads* no programa, a função *main* não pode simplesmente terminar, com *return* ou *exit*(), pois isso faria com que o processo todo, com todas as suas *threads*, fosse **terminado**.

Assim, é possível que a função *main*, que é a *thread* original associada a este processo, passe à execução de algum códido com duração equivalente à das demais *threads*. Também é comum que a função *main* seja usada apenas como uma coordenadora das atividades do programa. Neste caso, ela pode simplesmente esperar pela conclusão das demais *threads*.

Ao concluir suas atividades, uma função que corresponde ao código de uma *thread* pode realizar o **retorno de algum valor**, ou pode usar uma chamada explícita à função ***pthread_exit***(). Em ambos os casos, é possível retornar um ponteiro para alguma posição de memória.

Para efeitos de sincronização e/ou para saber o resultado da execução de uma outra *thread* do mesmo processo, é possível usar a chamada [***pthread_join***](https://man7.org/linux/man-pages/man3/pthread_join.3.html)(). De maneira parecida com a chamada *waitpid*(), *pthread_join* serve para esperar pelo retorno (conclusão) de uma *thread* específica.

Qualquer *thread* de um processo pode usar *pthread_join* para esperar por qualquer outra *thread* deste processo; isso não precisa ser feito pela função *main* ou pela *thread* que criou a *thread* que se quer esperar. Na chamada, contudo, é preciso fornecer o identificador da *thread* esperada.



O programa exemplo a seguir ilustra a criação de *threads* com *pthreads* (*POSIX threads*).

Compilando programas com *pthreads* com **gcc**

Só um comentário rápido aqui. Para compilar com **gcc** programas C que usem as funções da API ***pthreads***, é preciso incluir o parâmetro ***-pthread*** na linha de comando.

Ex:
```
$ gcc prog.c -o prog -pthread
```

Para saber mais sobre isso, vejam o [manual](https://man7.org/linux/man-pages/man1/gcc.1.html):

       -pthread
           Define additional macros required for using the POSIX threads
           library.  You should use this option consistently for both
           compilation and linking.

In [ ]:
%%writefile t1.c

/*
 * Objetivo:
   Observar a criação de threads e o término de suas execuções.
   pthread_exit() ou return indicam fim de thread.
   Main thread deve esperar threads terminarem, ou sair com pthread_exit().
   Se função main terminar (return, exit ou _exit), processo todo (todas as suas
   threads) termina.
 */

#include <pthread.h>
#include <stdio.h>
#include <string.h>
#include <unistd.h>

#define NUM_THREADS	4

void *
hello_w(void *arg)
{
  pthread_t tid = pthread_self();

  sleep(5);

	printf("Um alo^ da thread %lu, parte do processo %d\n",
        (long unsigned)tid, (int)getpid() );

	pthread_exit(NULL);
}

int main (int argc, char *argv[])
{
	int t, status;

  // vetor de pthread_t para salvamento dos identificadores das threads
	pthread_t threads[NUM_THREADS];

  printf("Processo %d vai criar threads...\n\n", getpid() );

	for (t=0; t < NUM_THREADS; t++) {

		status = pthread_create(&threads[t], NULL, hello_w, NULL);

		if (!status) {
      printf("main criou thread %d (%lu)\n",t,(long unsigned)threads[t]);
    } else {
			printf("Falha da criacao da thread %d (%d)\n",t,status);
			return (1);
		}
	}

// pthread_exit(NULL);
// return 0;
// sleep(30);

  // Se função main, ou qualquer thread, terminar com exit, processo todo termina
  // Em geral, main executa algo e, ao final, espera demais threads acabarem antes
  // de terminar o programa. Espera pela conclusão é feita com pthread_join()

  // loop de espera pelo término da execução das threads
  for (t=0; t < NUM_THREADS; t++) {

    // join ignorando o valor de retorno
    status = pthread_join(threads[t], NULL);

    if (! status) {
      printf("Thread %d (%lu) retornou\n",t,(long unsigned)threads[t]);
    } else{
      fprintf(stderr,"Erro em pthread_join (%d)\n",status);
      return (2);
    }
  }

  return 0;
}

Writing t1.c


In [ ]:
! if [ ! t1 -nt t1.c ]; then gcc t1.c -o t1 -pthread; fi
! ./t1

Processo 28683 vai criar threads...

main criou thread 0 (140353395222272)
main criou thread 1 (140353386829568)
main criou thread 2 (140353378436864)
main criou thread 3 (140353370044160)
Um alo^ da thread 140353386829568, parte do processo 28683
Um alo^ da thread 140353395222272, parte do processo 28683
Um alo^ da thread 140353370044160, parte do processo 28683
Um alo^ da thread 140353378436864, parte do processo 28683


**Questões**:

* Viram que todas as *threads* exibem o mesmo ***pid***?
* Viram os valores dos ***thread_ids***?
* Por que as impressões das mensagens das *threads* podem aparecer fora de ordem, em ordem diferente a cada execução?


O programa a seguir tem sentido? Por quê?

In [ ]:
%%writefile t2.c

#include <pthread.h>
#include <stdio.h>
#include <unistd.h>

#define NUM_THREADS	4

void *
hello_w(void *arg)
{
  printf("Thread %ld trabalhando...\n", (long int)pthread_self() );
  sleep(1);
	pthread_exit(NULL);
}

int main (int argc, char *argv[])
{
	int t;

  // vetor de pthread_t para salvamento dos identificadores das threads
	pthread_t threads[NUM_THREADS];

	for (t=0; t < NUM_THREADS; t++) {

 		if (pthread_create(&threads[t], NULL, hello_w, NULL)) {
      printf("Falha da criacao da thread %d\n",t);
			return(1);
		}

    if (pthread_join(threads[t], NULL) ) {
      printf("Erro em pthread_join, esperando pela thread %d\n",t);
      return(2);
    }
  }

  return 0;
}

Writing t2.c


In [ ]:
! if [ ! t2 -nt t2.c ]; then gcc -Wall t2.c -o t2 -pthread; fi
! ./t2

Thread 140661673895680 trabalhando...
Thread 140661673895680 trabalhando...
Thread 140661673895680 trabalhando...
Thread 140661673895680 trabalhando...


# Passando parâmetros na ativação e no retorno das *threads*

Vejamos como funciona a passagem de parâmetros para a função da *thread*, tanto em sua criação quanto no retorno da função associada.

Como visto, a chamada *pthread_create* prevê apenas um parâmetro na função da *thread*. Sendo este um ponteiro sem tipo definido (***void ****), em C, isso pode ser usado para passar ponteiros para qualquer posição de memória, que contenha qualquer tipo de dado!

Ah, considerando as arquiteturas 64 bits atuais, em que um endereço de memória tem 64 bits, também é possível passar como parâmetro para a função da *thread* qualquer valor que caiba em (<=) 8 bytes.

<br>

Uma estratégia para acomodar funções que precisam múltiplos parâmetros é definir um tipo de estrutura (***struct***) contendo esses parâmetros. Daí, para a chamada de criação da *thread*, preenche-se uma estrutura deste tipo com os parâmetros desejados e passa-se o endereço desta estrutura como parâmetro da chamada.

Nos casos de uso de *threads* para programação paralela com decomposição de dados, uma estratégia comum é passar para a função da *thread* apenas um identificador lógico desta *thread*. Esse identificador pode então ser usado para que a própria *thread* determine qual parte dos dados irá manipular.

<br>

Quanto ao valor de retorno, as mesmas considerações são válidas sobre o uso de um ponteiro. No código de chamada da função *pthread_join*, é preciso estar atento que, originialmente, deverá ser fornecido um endereço de uma posição de memória **onde será salvo um endereço** (void **).

Por fim, como a linguagem C é bastante flexível, basta que o programa passe parâmetros que são tratados de maneira apropriada nas funções que emitem e processam as chamadas.

In [ ]:
%%writefile t3.c

#include <pthread.h>
#include <stdio.h>
#include <string.h>
#include <unistd.h>

#define NUM_THREADS	4

// declare aqui, como variávies globais, as variáveis que serão
// compartilhadas entre as threads deste processo

void *
hello_w(void *arg)
{
  int * num = (int *)arg;
  int ind = *num;
  // int val = *(int *)arg;

	printf("Thread %d trabalhando...\n", ind);
  sleep(1);

	pthread_exit(NULL);
}

int main (int argc, char *argv[])
{
	int t, status;

  // vetor de pthread_t para salvamento dos identificadores das threads
	pthread_t threads[NUM_THREADS];

  // vetor de inteiros para uso como parâmetros para as threads
  int args[NUM_THREADS];

	for (t=0; t < NUM_THREADS; t++) {

    args[t]=t;  // ajuste do parâmetro para a thread; neste caso, um índice lógico

		status = pthread_create(&threads[t], NULL, hello_w, (void *)&args[t]);

		if (!status) {
      printf("main criou thread %d\n",t);
    } else {
			printf("Falha da criacao da thread %d (%d)\n",t,status);
			return (1);
		}
	}

  // loop de espera pelo término da execução das threads
  for (t=0; t < NUM_THREADS; t++) {

    // join ignorando o valor de retorno
    status = pthread_join(threads[t], NULL);

    if (! status) {
      printf("Thread %d (%lu) retornou\n",t,(long unsigned)threads[t]);
    } else{
      printf("Erro em pthread_join da thread %d (%d)\n",t,status);
      return (2);
    }
  }

  return 0;
}

Writing t3.c


In [ ]:
! if [ ! t3 -nt t3.c ]; then gcc -Wall t3.c -o t3 -pthread; fi
! ./t3

main criou thread 0
main criou thread 1
main criou thread 2
Thread 2 trabalhando...
main criou thread 3
Thread 0 trabalhando...
Thread 1 trabalhando...
Thread 3 trabalhando...
Thread 0 (140596413261568) retornou
Thread 1 (140596404868864) retornou
Thread 2 (140596396476160) retornou
Thread 3 (140596388083456) retornou


Criando diferentes tipos de *trheads*, com diferentes parâmetros...

In [ ]:
%%writefile t4.c
/*
** Objetivo:
    observar que o tipo *void usado no argumento da função da thread pode
    ser qualquer coisa. É preciso apenas que o argumento seja usado considerando
    o tipo que foi passado.
    Não é qualquer tipo de parâmetro que pode ser tratado como um ponteiro, contudo.
    No caso da estrutura, é preciso passar o seu endereço.
*/

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

#define LEN 64

// exemplo de estrutura de dados para usar como parâmetos na criação
// de thread usando função que precisa de vários parâmetros
typedef struct par {
	int num_int;
	char num_char[LEN];
} st_par;


// exemplo de função que espera um valor inteiro (long int) como parâmetro
void *
f_int(void *num_int)
{
	printf("Aloˆ, aqui é %ld\n",(long int)num_int);

	// pthread_exit(NULL);
	pthread_exit((long int *)1);
}

// exemplo de função que espera uma string (vetor de char) como parâmetro
void *
f_char(void *num_char)
{
	printf("Aqui é thread %s\n",(char *)num_char);

	// pthread_exit(NULL);
	pthread_exit((long int *)2);
}

// exemplo de função que precisa de vários parâmetros
void *
f_struct(void *num_struct)
{
	st_par *param;
	param=(st_par*)num_struct;
	// int *intp;
	// char *charp;

	// intp = &(param->num_int);
	// charp = &(param->num_char);

	printf("Ola' da thread %d %s\n",param->num_int,param->num_char);

	// pthread_exit(NULL);
	pthread_exit((long int *)34);
}

int
main (int argc, char *argv[])
{
	pthread_t th_int, th_char, th_struct;
	int status, *retval;
	long int num_int=1;
	char *num_char="dois";
	st_par num_struct;
	char err_msg[LEN];

	num_struct.num_int=3;
	strcpy(num_struct.num_char,"quatro");

	if((status=pthread_create(&th_int, NULL, f_int, (void *)num_int))) {
		strerror_r(status,err_msg,LEN);
		printf("Erro criando th_int: %s\n",err_msg);
		exit(EXIT_FAILURE);
	}
	if((status=pthread_create(&th_char, NULL, f_char, (void *)num_char))) {
		strerror_r(status,err_msg,LEN);
		printf("Erro criando th_char: %s\n",err_msg);
		exit(EXIT_FAILURE);
	}
	if((status=pthread_create(&th_struct, NULL, f_struct, (void *)&num_struct))) {
		strerror_r(status,err_msg,LEN);
		printf("Erro criando th_struct: %s\n",err_msg);
		exit(EXIT_FAILURE);
	}

	if((status = pthread_join(th_int, (void **)&retval))) {
		strerror_r(status,err_msg,LEN);
		printf("Erro em pthread_join th_int: %s\n",err_msg);
	}else
		printf("th_int joined: %ld\n",(long int)retval);

	if((status = pthread_join(th_char, (void **)&retval))) {
		strerror_r(status,err_msg,LEN);
		printf("Erro em pthread_join th_char: %s\n",err_msg);
	} else
		printf("th_char joined: %ld\n",(long int)retval);

	if((status = pthread_join(th_struct, (void **)&retval))) {
		strerror_r(status,err_msg,LEN);
		printf("Erro em pthread_join th_struct: %s\n",err_msg);
	} else
		printf("th_struct joined: %ld\n",(long int)retval);

	pthread_exit(NULL);
}

Writing t4.c


In [ ]:
! if [ ! t4 -nt t4.c ]; then gcc -Wall t4.c -o t4 -pthread; fi
! ./t4

Aloˆ, aqui é 1
Ola' da thread 3 quatro
th_int joined: 1
Aqui é thread dois
th_char joined: 2
th_struct joined: 34


# Compartilhando dados entre as *threads*

Um dos aspectos para o uso de *threads*, ao invés de processos, na criação de aplicações que querem explorar o uso de múltiplos processadores, é o compartilhamento de memória entre as *threads* de um processo.

De maneira simplificada, todas as **variáveis** que forem definidas como **globais**, ou seja, que forem **declaradas fora do escopo de qualquer função** no código, serão compartilhadas por todas as *threads* deste processo.

É claro, contudo, que se houver acessos simultâneos das *threads* sobre as mesmas posições de memória, pode ser preciso sincronizar esses acessos, evitando **condições de corrida** e erros. A garantida de acesso exclusivo, com exclusão mútua, por exemplo, não é provida automaticamente pelo SO. A aplicação deverá usar os mecanismos disponíveis para isso, se for o caso.


## Usando múltiplas *threads*

O programa a seguir ilustra o uso de *threads* para fazer a soma das linhas de 2 matrizes de forma paralela.

Para que sejam acessíveis por todas as *threads*, as matrizes foram declaradas globalmente.

Cabe uma discussão aqui, já que as matrizes serão alocadas dinamicamente.

Na verdade, as posições de memória contendo os endereços das matrizes, ou seja, os ponteiros, é que são globais. Neste exemplo, os espaços para as matrizes serão alocados dinamicamente. Vale saber, então, que as áreas de memória dinâmica (*heap*) também são acessíveis por todas as *threads* de um processo.

Neste exemplo específico, vale observar um aspecto relevante, que não é tratado no código, que é o cuidado com a divisão das linhas entre as *threads*. Para que o código seja flexível, com suporte a matrizes de tamanhos variados, ainda seria preciso tratar linhas restantes nos cálculos das divisões, para que não haja linhas sem manipulação!

In [ ]:
%%writefile t5.c
/*
 ** Programa :
 **   Uso de threads para soma das linhas de 2 matrizes
 */

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/time.h>
#include <sys/resource.h>MAP_SHARED            Modifications are shared.

#define LEN 128

#define NLIN 1000
#define NCOL 1000
#define NTHR 8

int *_A;
int *_B;
int *_C;

int _nlin = 0;
int _ncol = 0;
int _nthr = 0;

int _verbose = 0;

void *
soma(void *arg)
{
  // determina o número lógico desta thread
	long int ind = (long int)arg;
	int i,j;
	int nl, li, lf;

	// Falta tratar divisão não inteira...
	nl = _nlin/_nthr;
	li = ind*nl;
	lf = li+nl;

  if (_verbose)
	  printf("Thread %ld manipula linhas %d a %d\n",ind,li,lf-1);

	for(i=li; i < lf; i++)

		for(j=0;j<_ncol;j++)

			_C[i*_ncol+j] = _A[i*_ncol+j] + _B[i*_ncol+j];

	pthread_exit(NULL);
}

int
main (int argc, char *argv[])
{
	pthread_t threads[NLIN];
	long int t;
  int status;
	char err_msg[LEN];
	int i,j,ind;

	ind=1;
	while (ind < argc) {
		if( !strcmp(argv[ind],"-h") || !strcmp(argv[ind],"/?")) {
			printf("Uso: %s [-nc num_col] [-nl num_lin] [-nt num_thr] ",argv[0]);
			exit(0);
		}
    if(!strcmp(argv[ind],"-v"))
		  _verbose=1;

		if(!strcmp(argv[ind],"-nc")) {
			if(argc>ind)
				_ncol=atoi(argv[++ind]);
			else {
				printf("Erro nos parâmetros...\n");
				exit(0);
			}
		}
		if(!strcmp(argv[ind],"-nl")) {
			if(argc>ind)
				_nlin=atoi(argv[++ind]);
			else {
				printf("Erro nos parâmetros...\n");
				exit(0);
			}
		}
		if(!strcmp(argv[ind],"-nt")) {
			if(argc>ind)
				_nthr=atoi(argv[++ind]);
			else {
				printf("Erro nos parâmetros...\n");
				exit(0);
			}
		}

		ind++;
	}MAP_SHARED            Modifications are shared.

	if(_nlin==0) _nlin = NLIN;
	if(_ncol==0) _ncol = NCOL;
	if(_nthr==0) _nthr = NTHR;

	// alocar as matrizes
	_A=(int *)malloc(_nlin * _ncol * sizeof(int));
	_B=(int *)malloc(_nlin * _ncol * sizeof(int));
	_C=(int *)malloc(_nlin * _ncol * sizeof(int));

  srand(getpid());

  // geração das matrizes com valores inteiros aleatórios (0 a 9). Senão, 0!
	for(i=0;i<_nlin;i++) {
		for(j=0;j<_ncol;j++) {
			_A[i*_ncol+j]=rand()%10;
			_B[i*_ncol+j]=rand()%10;
		}
	}

	if(_verbose) {
		printf("\n");
		for(i=0;i<_nlin;i++) {
			printf("%d: ",i);
			for(j=0;j<_ncol;j++)
				printf("%d ",_A[i*_ncol+j]);
			printf("   ");
			for(j=0;j<_ncol;j++)
				printf("%d ",_B[i*_ncol+j]);
			printf("\n");
		}
    printf("\n");
	}

  // cria threads, passando o índice lógico como parâmetro para cada uma delas
	for (t=0; t<_nthr; t++) {
		status = pthread_create(&threads[t], NULL, soma, (void *)t);
		if (status) {
			strerror_r(status,err_msg,LEN);
			printf("Falha da criacao da thread %ld (%d): %s\n",t,status,err_msg);
			exit(EXIT_FAILURE);
		}
	}

	// espera threads retornarem
	for (t=0; t<_nthr; t++) {

		status = pthread_join(threads[t], NULL);
		if (status) {
			strerror_r(status,err_msg,LEN);
			printf("Erro em pthread_join: %s\n",err_msg);
			exit(EXIT_FAILURE);
		}
	}

	// impressão da matriz resultante
	if(_verbose) {
		printf("\n");
		for(i=0;i<_nlin;i++) {
			printf("%d: ",i);
			for(j=0;j<_ncol;j++)
				printf("%2d ",_C[i*_nlin+j]);
			printf("\n");
		}
		printf("\n");
	}

	// desalocar as matrizes
	free(_A);
	free(_B);
	free(_C);

	return(0);
}

Writing t5.c


In [ ]:
! if [ ! t5 -nt t5.c ]; then gcc -Wall t5.c -o t5 -pthread ; fi
# ! time ./t5 -nl 10 -nc 10 -nt 5 -v
! lscpu | grep "CPU(s):"
! time ./t5 -nl 10240 -nc 10240 -nt 1
! time ./t5 -nl 10240 -nc 10240 -nt 2
! time ./t5 -nl 10240 -nc 10240 -nt 4
! time ./t5 -nl 10240 -nc 10240 -nt 8
! time ./t5 -nl 10240 -nc 10240 -nt 16
! time ./t5 -nl 10240 -nc 10240 -nt 32

CPU(s):                          2
NUMA node0 CPU(s):               0,1

real	0m3.088s
user	0m2.548s
sys	0m0.523s

real	0m3.006s
user	0m2.861s
sys	0m0.568s

real	0m4.002s
user	0m3.491s
sys	0m0.615s

real	0m2.979s
user	0m2.864s
sys	0m0.546s

real	0m3.090s
user	0m2.942s
sys	0m0.568s

real	0m3.027s
user	0m2.870s
sys	0m0.574s
